In [1]:
import torch
import torch.nn as nn
import  pandas as pd
import  numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sympy.abc import epsilon
from torchinfo import summary
from sklearn.model_selection import  train_test_split

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [4]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [5]:
X = df.drop(columns='diagnosis')
y = df['diagnosis']

In [6]:
X_train , X_test, y_train , y_test = train_test_split(X,y , test_size=0.2,random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [9]:
X_train_tensor = torch.from_numpy(X_train_scaled).float()
X_test_tensor  = torch.from_numpy(X_test_scaled).float()

y_train_tensor = torch.from_numpy(y_train_encoded).float()
y_test_tensor  = torch.from_numpy(y_test_encoded).float()


In [10]:
class model0(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()

    def forward (self,features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out



    def loss_functn(self,y_pred,y):
        epsilon = 1e-7
        y_pred= torch.clamp(y_pred,epsilon,1-epsilon)

        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()


        return loss

In [11]:
learning_rate = 0.1
epoch = 25

In [12]:
epochs = 25
learning_rate = 0.1

mod0 = model0(X_train_tensor.shape[1])

for ep in range(epochs):
    y_pred = mod0(X_train_tensor)
    loss = mod0.loss_functn(y_pred, y_train_tensor)

    loss.backward()

    with torch.no_grad():
        mod0.linear.weight -= learning_rate * mod0.linear.weight.grad
        mod0.linear.bias   -= learning_rate * mod0.linear.bias.grad

    mod0.linear.weight.grad.zero_()
    mod0.linear.bias.grad.zero_()

    print(f"Epoch: {ep+1}, Loss: {loss.item():.4f}")


Epoch: 1, Loss: 0.7306
Epoch: 2, Loss: 0.7239
Epoch: 3, Loss: 0.7189
Epoch: 4, Loss: 0.7150
Epoch: 5, Loss: 0.7117
Epoch: 6, Loss: 0.7087
Epoch: 7, Loss: 0.7061
Epoch: 8, Loss: 0.7036
Epoch: 9, Loss: 0.7014
Epoch: 10, Loss: 0.6993
Epoch: 11, Loss: 0.6973
Epoch: 12, Loss: 0.6954
Epoch: 13, Loss: 0.6937
Epoch: 14, Loss: 0.6921
Epoch: 15, Loss: 0.6905
Epoch: 16, Loss: 0.6890
Epoch: 17, Loss: 0.6877
Epoch: 18, Loss: 0.6863
Epoch: 19, Loss: 0.6851
Epoch: 20, Loss: 0.6839
Epoch: 21, Loss: 0.6828
Epoch: 22, Loss: 0.6817
Epoch: 23, Loss: 0.6807
Epoch: 24, Loss: 0.6798
Epoch: 25, Loss: 0.6789


In [13]:
with torch.no_grad():
  y_pred = mod0.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6228070259094238


In [14]:
loss_fn = nn.BCELoss()


In [15]:
epochs = 25
learning_rate = 0.1

mod1 = model0(X_train_tensor.shape[1])

for ep in range(epochs):
    y_pred = mod1(X_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor.view(-1,1))

    loss.backward()

    with torch.no_grad():
        mod1.linear.weight -= learning_rate * mod1.linear.weight.grad
        mod1.linear.bias   -= learning_rate * mod1.linear.bias.grad

    mod1.linear.weight.grad.zero_()
    mod1.linear.bias.grad.zero_()

    print(f"Epoch: {ep+1}, Loss: {loss.item():.4f}")


Epoch: 1, Loss: 0.5364
Epoch: 2, Loss: 0.4424
Epoch: 3, Loss: 0.3850
Epoch: 4, Loss: 0.3459
Epoch: 5, Loss: 0.3174
Epoch: 6, Loss: 0.2955
Epoch: 7, Loss: 0.2780
Epoch: 8, Loss: 0.2637
Epoch: 9, Loss: 0.2517
Epoch: 10, Loss: 0.2415
Epoch: 11, Loss: 0.2326
Epoch: 12, Loss: 0.2249
Epoch: 13, Loss: 0.2180
Epoch: 14, Loss: 0.2118
Epoch: 15, Loss: 0.2063
Epoch: 16, Loss: 0.2012
Epoch: 17, Loss: 0.1966
Epoch: 18, Loss: 0.1924
Epoch: 19, Loss: 0.1885
Epoch: 20, Loss: 0.1849
Epoch: 21, Loss: 0.1816
Epoch: 22, Loss: 0.1785
Epoch: 23, Loss: 0.1756
Epoch: 24, Loss: 0.1728
Epoch: 25, Loss: 0.1703


In [16]:
with torch.no_grad():
  y_pred = mod1.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5667898058891296


In [17]:
epochs = 25
learning_rate = 0.1

mod2 = model0(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(mod2.parameters(), lr=learning_rate)

for ep in range(epochs):
    y_pred = mod2(X_train_tensor)

    loss = loss_fn(y_pred, y_train_tensor.view(-1, 1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch: {ep+1}, Loss: {loss.item():.4f}")


Epoch: 1, Loss: 0.8226
Epoch: 2, Loss: 0.6058
Epoch: 3, Loss: 0.4909
Epoch: 4, Loss: 0.4222
Epoch: 5, Loss: 0.3763
Epoch: 6, Loss: 0.3430
Epoch: 7, Loss: 0.3175
Epoch: 8, Loss: 0.2973
Epoch: 9, Loss: 0.2808
Epoch: 10, Loss: 0.2669
Epoch: 11, Loss: 0.2551
Epoch: 12, Loss: 0.2449
Epoch: 13, Loss: 0.2360
Epoch: 14, Loss: 0.2281
Epoch: 15, Loss: 0.2210
Epoch: 16, Loss: 0.2147
Epoch: 17, Loss: 0.2090
Epoch: 18, Loss: 0.2037
Epoch: 19, Loss: 0.1990
Epoch: 20, Loss: 0.1946
Epoch: 21, Loss: 0.1905
Epoch: 22, Loss: 0.1868
Epoch: 23, Loss: 0.1833
Epoch: 24, Loss: 0.1800
Epoch: 25, Loss: 0.1770


In [18]:
with torch.no_grad():
  y_pred = mod2.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5624807476997375
